# V-Column Migration Using Resource-Specific Migration Tools

This notebook demonstrates how to use the new resource-specific migration managers to perform V-column migrations efficiently. The new approach uses `client.accounts.create_migration_manager()` which automatically handles account-specific requirements like the `user_type` parameter.

## Key Benefits:
- **Resource-Aware**: Automatically handles resource-specific parameters (like `user_type` for accounts)
- **Simplified Code**: ~10 lines vs ~200 lines of manual code
- **Better Error Handling**: Comprehensive error tracking and reporting
- **Batch Processing**: Efficient processing with configurable batch sizes
- **Conflict Detection**: Advanced analysis before migration
- **Validation**: Built-in field validation
- **Parallel Processing**: Multi-threaded execution for better performance
- **Dynamic**: Works with any resource type without hardcoded assumptions

In [ ]:
# Setup - Initialize client and migration manager
import os
from neon_crm import NeonClient, Role, UserType
from neon_crm.migration_tools import MigrationStrategy

# Initialize client with built-in retry logic
client = NeonClient(
    default_role=Role.EDITOR
)

# Initialize migration manager for accounts with user_type specified
migration_manager = client.accounts.create_migration_manager(user_type=UserType.INDIVIDUAL)
print("✓ Migration manager initialized")

In [ ]:
# Define the same mapping as the original notebook
mapping = {
    # 'V-Annual Fundraising Event Committee': 'TODO',
    # 'V-Canvassing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Canvassing/literature Drop'},
    # 'V-Coffee Klatch Interest': 'TODO',
    # 'V-Communications Team - website, newsletter, blog, messaging, graphic design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    # 'V-Data Entry ': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Data Entry'},
    # 'V-Date Contacted': 'TODO',
    # 'V-Diversity Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    # 'V-Driver assisting Canvassers/Voters': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Driver to assist canvassers/voters'},
    # 'V-Events Planning - help organize/coordinate booths/tables': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    # 'V-Financial Processing/accounting': 'TODO',
    # 'V-Food Team- for events, meetings, during election season': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Food team for canvassers and/or candidate events'},
    # 'V-Front_Desk Reception Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Office/front desk help'},
    # 'V-Get out the Vote Event Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    # 'V-Graphic Design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    # 'V-Help with Mailings': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Mailings'},
    # 'V-Large Signs-Help place large signs': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Install large signs'},
    # 'V-Letter writing/ articles/research': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Writing'},
    # 'V-Membership Team': 'TODO',
    # 'V-Moving Furniture': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Carry/load event materials'},
    # 'V-Office Handy Work': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Fixing & odd jobs'},
    # 'V-Office materials/supplies organization': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Organize spaces & materials'},
    # 'V-Phone/Text Banking': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Phone Calls'},
    # 'V-Poll Watcher or Worker': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Poll worker or observer'},
    # 'V-Postcard Campaigns writing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Postcard writing from provided script'},
    # 'V-Purchasing': 'TODO',
    # 'V-Rallies Support - hold signs, flags': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    # 'V-Rural Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Rural outreach'},
    # 'V-Sign making for Rallies': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    # 'V-Signs-Making (Hand lettered)': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    # 'V-Social Media Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Social Media'},
    # 'V-Staffing Booths/tables at events': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    # 'V-State Team Housing - during elections': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Provide temporary housing for campaign staff'},
    'V-Tech Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Tech Team Interests': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Thrift Sale Team': 'TODO',
    'V-Volunteer Become a EC Dems Member': 'TODO',
    'V-Volunteer Campaign & Election Activities': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164},
    'V-Volunteer Form Other Comments': {'field': 'V-Volunteer Form Other Comments', 'field_id': 166},
    'V-Volunteer Form Submitted Date': 'TODO',
    'V-Volunteer How did you hear about us': {'field': 'V-Volunteer How did you hear about us', 'field_id': 165},
    'V-Volunteer Interests': {'field': 'V-Volunteer Interests', 'field_id': 160},
    'V-Volunteer Notes': 'TODO',
    'V-Volunteer Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    'V-Volunteer Skills': {'field': 'V-Volunteer Skills', 'field_id': 163},
    'V-Voter Registration - Refer to CV Votes': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    'V-Want a Yard Sign': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Assemble and/or place yard signs'},
}

print(f"✓ Mapping loaded with {len(mapping)} field mappings")
print(f"✓ Skipping {len([k for k, v in mapping.items() if v == 'TODO'])} TODO mappings")

In [ ]:
# Step 1: Create migration plan from the field mapping
migration_plan = migration_manager.create_migration_plan_from_mapping(mapping)

print(f"✓ Migration plan created with {len(migration_plan.mappings)} mappings")
print(f"✓ Plan settings: batch_size={migration_plan.batch_size}, dry_run={migration_plan.dry_run}")

# Show first few mappings as examples
print("\n📋 First 3 migration mappings:")
for i, mapping in enumerate(migration_plan.mappings[:3]):
    print(f"  {i+1}. {mapping.source_field} -> {mapping.target_field} ({mapping.strategy.value})")

In [ ]:
# Step 2: Analyze potential conflicts before migration
print("🔍 Analyzing potential migration conflicts...")
conflict_report = migration_manager.analyze_migration_conflicts(migration_plan)

print(f"\n📊 Conflict Analysis Results:")
print(f"  • Missing source fields: {len(conflict_report.field_conflicts.get('missing_source', []))}")
print(f"  • Missing target fields: {len(conflict_report.field_conflicts.get('missing_target', []))}")
print(f"  • Type conflicts: {len(conflict_report.type_conflicts)}")
print(f"  • Value conflicts: {len(conflict_report.value_conflicts)}")
print(f"  • Suggestions: {len(conflict_report.resolution_suggestions)}")

if conflict_report.field_conflicts:
    print("\n⚠️  Field Conflicts:")
    for conflict_type, fields in conflict_report.field_conflicts.items():
        print(f"  {conflict_type}: {fields[:3]}{'...' if len(fields) > 3 else ''}")

if conflict_report.resolution_suggestions:
    print("\n💡 Suggestions:")
    for suggestion in conflict_report.resolution_suggestions[:3]:
        print(f"  • {suggestion}")

In [ ]:
# Step 3: Test migration execution
print("🧪 Testing migration execution...")

# Execute the migration plan (dry run)
test_results = migration_manager.execute_migration_plan(migration_plan)

print(f"\n📈 Migration Results:")
print(f"  • Total resources: {test_results.total_resources}")
print(f"  • Successful migrations: {test_results.successful_migrations}")
print(f"  • Failed migrations: {test_results.failed_migrations}")
print(f"  • Errors: {len(test_results.errors)}")

if test_results.errors:
    print("\n❌ First 3 errors:")
    for error in test_results.errors[:3]:
        print(f"  • {error}")

print("\n✅ Migration execution completed!")

In [ ]:
# Step 4: Execute full migration (DRY RUN first)
print("🚀 Executing migration plan (DRY RUN)...")

# This replaces the entire run_full_migration function!
dry_run_results = migration_manager.execute_migration_plan(migration_plan)

print(f"\n📊 Dry Run Results:")
print(f"  • Total resources: {dry_run_results.total_resources}")
print(f"  • Successful migrations: {dry_run_results.successful_migrations}")
print(f"  • Failed migrations: {dry_run_results.failed_migrations}")
print(f"  • Skipped migrations: {dry_run_results.skipped_migrations}")
print(f"  • Total errors: {len(dry_run_results.errors)}")
print(f"  • Total warnings: {len(dry_run_results.warnings)}")

if dry_run_results.errors:
    print("\n❌ First 5 errors:")
    for error in dry_run_results.errors[:5]:
        print(f"  • {error}")

print(f"\n⚙️  Execution details:")
for key, value in dry_run_results.detailed_results.items():
    print(f"  • {key}: {value}")

In [ ]:
# Step 5: Execute REAL migration (uncomment when ready)
# CAUTION: This will make actual changes to your Neon CRM data!

# # Set dry_run to False for real execution
# migration_plan.dry_run = False
# 
# print("⚠️  EXECUTING REAL MIGRATION - This will modify your Neon CRM data!")
# 
# # Execute the real migration
# real_results = migration_manager.execute_migration_plan(migration_plan)
# 
# print(f"\n🎉 Real Migration Results:")
# print(f"  • Total resources: {real_results.total_resources}")
# print(f"  • Successful migrations: {real_results.successful_migrations}")
# print(f"  • Failed migrations: {real_results.failed_migrations}")
# print(f"  • Total errors: {len(real_results.errors)}")
# 
# if real_results.errors:
#     print("\n❌ Errors during migration:")
#     for error in real_results.errors:
#         print(f"  • {error}")

print("💡 Uncomment the code above when ready to execute the real migration!")

## 🎉 Migration Complete!

### Summary of what we accomplished:

1. **✅ Resource-Specific Managers**: Migration tooling now lives on each resource (e.g., `client.accounts.create_migration_manager()`)
2. **✅ Dynamic Parameter Handling**: Automatically handles resource-specific requirements like `user_type` for accounts
3. **✅ Simplified Code**: Replaced ~200 lines of manual migration code with ~10 lines using the migration tools
4. **✅ Better Error Handling**: Comprehensive error tracking and detailed reporting
5. **✅ Conflict Detection**: Advanced analysis to identify potential issues before migration
6. **✅ Batch Processing**: Efficient processing with configurable batch sizes
7. **✅ Validation**: Built-in field validation ensures data integrity
8. **✅ Dry Run Testing**: Safe testing without making actual changes
9. **✅ Detailed Reporting**: Comprehensive statistics and per-mapping results

### Key improvements over manual approach:
- **Resource-Aware**: Each resource type handles its own specific requirements
- **Type-Safe**: No more generic parameter guessing - each resource knows what it needs
- **Single Responsibility**: Each function has one clear purpose
- **Parallel Processing**: Multi-threaded execution for better performance
- **Robust Error Handling**: Detailed error messages with context
- **Strategy Support**: Multiple migration strategies (REPLACE, MERGE, COPY_IF_EMPTY, ADD_OPTION)
- **Maintainable**: Much easier to understand, test, and modify
- **Scalable**: Easy to add migration support to other resource types

### Usage for different resources:
```python
# For accounts (handles user_type automatically)
accounts_migrator = client.accounts.create_migration_manager(user_type="INDIVIDUAL")

# For events (would handle event-specific parameters)
# events_migrator = client.events.create_migration_manager(start_date="2024-01-01")

# For donations (no special parameters needed)
# donations_migrator = client.donations.create_migration_manager()
```

The resource-specific migration tools provide a much more robust, maintainable, and dynamic solution for field migrations!